In [1]:
print("hello world")

hello world


In [3]:
# Setup and imports
# STEP 1: Install and import required packages (run this first!)
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

import folium
import pandas as pd
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

print("All packages installed and imported successfully!")



All packages installed and imported successfully!


In [4]:
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df = pd.read_csv(spacex_csv_file)

spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

print("Data loaded successfully!")


Data loaded successfully!


## Task 1: Mark all launch sites on a map


In [6]:
# Initialize the map
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add markers for each launch site
for lat, lng, label in zip(launch_sites_df['Lat'], launch_sites_df['Long'], launch_sites_df['Launch Site']):
    coordinate = [lat, lng]
    folium.Circle(coordinate, radius=1000, color='#3186cc', fill=True).add_child(folium.Popup(label)).add_to(site_map)
    folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#3186cc;"><b>{label}</b></div>',
        )
    ).add_to(site_map)

site_map



## Task 2: Mark the success/failed launches for each site on the map

In [20]:
# Function to assign colors
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'  # Success
    else:
        return 'red'    # Failure

# Apply color assignment
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Create marker cluster
marker_cluster = MarkerCluster()

# Add markers for each launch
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']], 
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map.add_child(marker_cluster)
site_map


## TASK 3: Calculate the distances between a launch site to its proximities

In [7]:
# TASK 3: Calculate distances (NOW IT WILL WORK!)

# Add mouse position plugin
mouse_position = MousePosition(position='topright', prefix='Coordinates:')
site_map.add_child(mouse_position)

# Distance calculation function
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

# Calculate distance
ksc_lat, ksc_lon = 28.573255, -80.648895
coastline_lat, coastline_lon = 28.56367, -80.57163
distance = calculate_distance(ksc_lat, ksc_lon, coastline_lat, coastline_lon)

# Add markers and lines
folium.Marker([coastline_lat, coastline_lon], icon=folium.Icon(color='red')).add_to(site_map)
folium.PolyLine([[ksc_lat, ksc_lon], [coastline_lat, coastline_lon]], color='red').add_to(site_map)

print(f"Distance to coastline: {distance:.2f} km")
site_map


Distance to coastline: 7.62 km
